<table align="left"><tr><td>
<a href="https://colab.research.google.com/github/kikim6114/NLP2024-1/blob/main/06_summarization-kikim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="코랩에서 실행하기"/></a>
</td></tr></table>

이 노트북을 코랩에서 실행하려면 Pro 버전이 필요할 수 있습니다.

In [1]:
# 코랩을 사용하지 않으면 다음 코드를 주석 처리하세요.
# !git clone https://github.com/kikim6114/nlp-practice-2025.git
# %cd nlp-practice-2025
# from install import *
# install_requirements(chapter=6)

Cloning into 'nlp-practice-2025'...
remote: Enumerating objects: 205, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 205 (delta 11), reused 11 (delta 10), pack-reused 190 (from 1)
Receiving objects: 100% (205/205), 29.30 MiB | 16.28 MiB/s, done.
Resolving deltas: 100% (52/52), done.
/content/nlp-practice-2025
⏳ Installing base requirements ...
✅ Base requirements installed!
Using transformers v4.51.3
Using datasets v2.14.4
Using accelerate v1.6.0
Using sentencepiece v0.2.0
Using sacrebleu v2.5.1
Using rouge_score
Using nltk v3.9.1
Using py7zr v0.22.0


In [4]:
from transformers import pipeline, set_seed

# 5. 요약

요약(Summarization)이 도전적 작업인 이유:
- 긴 단락의 이해
- 관련 내용의 추론
- 원래 문서의 주제를 통합해 유창한 텍스트를 생성
- 영역별 요약 방법이 다르다(예: 기사와 법률계약서의 요약 방법은 다르다)

## 5.1 CNN/DailyMail 데이터셋

CNN/DailyMail dataset
- 300,000개 뉴스 기사와 요약의 쌍
- 요약을 위해 익명화 처리되지 않은 v3.0.0 사용
- 요약: 본문에서 추출(extractive)하지 않고 추상적(abstractive)이다.
  - Extractive: 본문의 어떤 구간을 가져다 사용함
  - Abstractive: 내용에 맞춰 글을 생성

In [ ]:
# !pip install -U datasets

In [6]:
from datasets import load_dataset, config
# !rm -rf {config.HF_DATASETS_CACHE}
dataset = load_dataset("ccdv/cnn_dailymail", version="3.0.0", trust_remote_code=True)
# dataset = load_dataset("ccdv/cnn_dailymail", version="3.0.0")  # original
print(f"특성: {dataset['train'].column_names}")

cnn_stories.tgz:   0%|          | 0.00/159M [00:00<?, ?B/s]

dailymail_stories.tgz:   0%|          | 0.00/376M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

특성: ['article', 'highlights', 'id']


In [7]:
sample = dataset["train"][1]
print(f"""기사 (500개 문자 발췌, 총 길이: {len(sample["article"])}):""")
print(sample["article"][:500])
print(f'\n요약 (길이: {len(sample["highlights"])}):')
print(sample["highlights"])

기사 (500개 문자 발췌, 총 길이: 3192):
(CNN) -- Usain Bolt rounded off the world championships Sunday by claiming his third gold in Moscow as he anchored Jamaica to victory in the men's 4x100m relay. The fastest man in the world charged clear of United States rival Justin Gatlin as the Jamaican quartet of Nesta Carter, Kemar Bailey-Cole, Nickel Ashmeade and Bolt won in 37.36 seconds. The U.S finished second in 37.56 seconds with Canada taking the bronze after Britain were disqualified for a faulty handover. The 26-year-old Bolt has n

요약 (길이: 180):
Usain Bolt wins third gold of world championship .
Anchors Jamaica to 4x100m relay victory .
Eighth gold at the championships for Bolt .
Jamaica double up in women's 4x100m relay .


기사가 긴 경우, 입력 시퀀스 최대 길이가 1024인 Transformer 특성상 뒷부분의 기사에 포함된 정보는 사라진다.

## 6.2 텍스트 요약 파이프라인
- 여러 트랜스포머 기반 요약 모델을 살펴보자.
- 비교를 위해 입력의 길이가 모두 동일하도록 길이를 2000 글자로 제한한다.

In [8]:
sample_text = dataset["train"][1]["article"][:2000]
# 딕셔너리에 각 모델이 생성한 요약을 저장합니다.
summaries = {}

- 텍스트를 문장 단위로 분리하는 것이 관행이므로,
- 좋은 성능을 갖는 NLTK 패키지를 사용하여 분할한다.

In [11]:
import nltk
from nltk.tokenize import sent_tokenize

nltk.download("punkt")
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kikim\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\kikim\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


True

In [12]:
string = "The U.S. are a country. The U.N. is an organization."
sent_tokenize(string)

['The U.S. are a country.', 'The U.N. is an organization.']

### 6.2.1 요약 기준 모델

- 요약된 텍스트의 처음 3 문장만 가져오는 함수를 정의
- 문장 구분은 `\n` 문자

In [13]:
def three_sentence_summary(text):
    return "\n".join(sent_tokenize(text)[:3])

In [14]:
summaries["baseline"] = three_sentence_summary(sample_text)
summaries

{'baseline': "(CNN) -- Usain Bolt rounded off the world championships Sunday by claiming his third gold in Moscow as he anchored Jamaica to victory in the men's 4x100m relay.\nThe fastest man in the world charged clear of United States rival Justin Gatlin as the Jamaican quartet of Nesta Carter, Kemar Bailey-Cole, Nickel Ashmeade and Bolt won in 37.36 seconds.\nThe U.S finished second in 37.56 seconds with Canada taking the bronze after Britain were disqualified for a faulty handover."}

### 6.2.2 GPT-2
- 입력 텍스트 끝에 `TL;DR`을 붙여서 요약을 생성하게 할 수 있다.
- `TL;DR`: Too Long; Didn't Read (Reddit 같은 사이트에서 긴 글을 중간에 생략하는 방법으로 사용)
- sample_text + "\nTL;DR:\n" 을 입력하여 🤗Transformers의 `pipeline()`으로 원래 기사를 재생성하는 것으로 실험을 시작한다는 것에 주목하자.

In [16]:
from transformers import pipeline, set_seed

set_seed(42)

# 코랩의 경우 gpt2-xl을 사용하면 메모리 부족 에러가 발생합니다.
# 대신 "gpt" 또는 "gpt2-large"로 지정하거나 코랩 프로를 사용하세요.
pipe = pipeline("text-generation", model="gpt2-large")

gpt2_query = sample_text + "\nTL;DR:\n"  # 프롬프트 형식을 만든다.
pipe_out = pipe(gpt2_query, max_length=512, clean_up_tokenization_spaces=True)
summaries["gpt2"] = "\n".join(
    sent_tokenize(pipe_out[0]["generated_text"][len(gpt2_query) :]))

config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

C:\Users\kikim\anaconda3\envs\nlp\lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\kikim\.cache\huggingface\hub\models--gpt2-large. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/3.25G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


### 6.2.3 T5

<img alt="T5" width="700" caption="Diagram of T5's text-to-text framework (courtesy of Colin Raffel); besides translation and summarization, the CoLA (linguistic acceptability) and STSB (semantic similarity) tasks are shown" src="images/chapter06_T5.png" id="T5"/>

- T5: 모든 NLP task들을 text-to-tex 작업으로 구성하는 universal transformer 아키텍처
- T5는 요약을 포함하는 지도 및 비지도 학습데이터로 훈련됨
- Fine-tuning 없이 pretraining에 사용했던 프롬프트를 사용해 바로 요약, 번역 등의 작업에 사용할 수 있다.
  - 요약 format: "summarize: \<ARTICLE\>"
  - 번역 format: "translate English to German: \<TEXT\>"
- 함수 `pipeline("summarization", model="t5-large")`는 입력을 text-to-text format으로 처리하므로 "summarize"를 붙일 필요가 없다.

In [ ]:
pipe = pipeline("summarization", model="t5-large")
pipe_out = pipe(sample_text)
summaries["t5"] = "\n".join(sent_tokenize(pipe_out[0]["summary_text"]))

### 6.2.4 BART

- 트랜스포머를 기반으로한 seq2seq 모델(encoder-decoder)로 구축된 denoising auto encoder
- BERT와 GPT-2 사전훈련 방식을 결합하여 훈련
- 사전학습 중에 BERT 류의 이전 모델들 보다 더 광범위한 노이즈 체계를 제공

<img alt="T5" width="500" src="images/chapter06_bart.png" id="bart">

- 여기서는 `CNN/DailyMail`에 미세조정된 `facebook/bart-large-ccn` checkpoint를 사용

In [ ]:
pipe = pipeline("summarization", model="facebook/bart-large-cnn")
pipe_out = pipe(sample_text)
summaries["bart"] = "\n".join(sent_tokenize(pipe_out[0]["summary_text"]))

### 6.2.5 PEGASUS
- Zhang, Jingqing, et al. "Pegasus: Pre-training with extracted gap-sentences for abstractive summarization." International conference on machine learning. PMLR, 2020.

<img alt="pegasus" width="700" caption="Diagram of PEGASUS architecture (courtesy of Jingqing Zhang et al.)" src="images/chapter06_pegasus.png" id="pegasus"/>

- 논문 제목에서 알 수 있듯이 Abstractive Summarization에 특화된 모델
- BART와 마찬가지로 Encoder-Decoder 트랜스포머
- Token 단위가 아닌 `Important Sentence` 단위로 masking 하고 이를 생성하는 방법(Gap Sentence Generation; GSG)으로 학습.
- Important sentence: Document 내에서 다른 문장에 비해 전체적인 context를 잘 설명할 수 있는 문장.

In [ ]:
pipe = pipeline("summarization", model="google/pegasus-cnn_dailymail")
pipe_out = pipe(sample_text)
summaries["pegasus"] = pipe_out[0]["summary_text"].replace(" .<n>", ".\n")

- 이 모델은 자동 줄바꿈하는 특수 토큰이 있어 `sent_tokenize()` 함수가 필요 없음

## 6.3 요약 결과 비교하기

- GPT-2는 데이터셋에서 전혀 훈련되지 않았음을 주목.
- T5는 여러 작업 중의 하나로 이 작업을 위해 미세조정됨.
- BART와 PEGASUS는 이 작업만을 위해 미세조정됨.

In [ ]:
print("GROUND TRUTH")
print(dataset["train"][1]["highlights"])
print("")

for model_name in summaries:
    print(model_name.upper())
    print(summaries[model_name])
    print("")

- GPT-2: **Hallucination**, **invented facts**
- 나머지 3개 모델의 요약을 정답과 비교하면 놀랄 정도로 많이 중복되며, 그중에서 PEGASUS가 정답에 가장 가깝다.

실 사용 환경에 사용할 모델의 선택:
- 몇 개의 샘플을 추가로 요약해 보는 것은 최선의 모델을 선택하는은 체계적 방법이 아니다.
- 지표 하나를 정의하고 어떤 벤치마크 데이터셋에서 모든 모델을 평가해서 성능이 최고인 모델을 선택하는 것이 이상적이 방법이다.
- 어떤 지표를 정의해야 텍스트 생성에 우수한 모델을 판별할 수 있을까?
  - accuracy, recall, 그리고 precision 같은 지표는 이 작업에 적용하기 어렵다
  - 사람이 작성한 'Gold standard' 요약마다 동의어, 다른 말로 바꿔 쓰거나, 또는 조금 다르게 작성하는 방식으로 수십개의 요약이 가능하기 때문.

## 6.4 생성된 텍스트 품질 평가하기

- 텍스트 생성 작업의 평가는 표준적인 분류 작업만큼 쉽지 않다.
- 단순히 참조 문장과 후보 문장이 정확히 일치하는지 확인하는 것은 최선이 아니다.
- 사람이 직접 작성한 "Gold Standard" 정답도 여러개일 수 있기 때문이다.

### 6.4.1 BLEU

- BLEU: Bilingual Evaluation Understudy
- Papineni, Kishore, et al. "Bleu: a method for automatic evaluation of machine translation." Proceedings of the 40th annual meeting of the Association for Computational Linguistics. 2002.
- 한 개의 생성된 후보 문장 $sent$ 와 (복수의) 참조 문장 $sent'$ 사이에, 𝑛-gram들이 얼마나 일치하는지를 비교.
- 비교 방법: 유사도(modified 𝑛-gram precision)
- Modified 𝑛-gram Precision:
$$p_n=\frac{\sum_{snt \in C}\sum_{n\mathrm{gram} \in snt'}Count_{\mathrm{clip}}(n\mathrm{gram})}{\sum_{snt' \in C}\sum_{n\mathrm{gram} \in snt}Count(n\mathrm{gram})}$$
- Brevity 페널티 (BR)
$$BR=\mathrm{min}\left(1, e^{1-l_{ref}/l_{gen}}\right)$$
- BLEU Score:  
$$\mathrm{BLEU}_N=BR \times \left(\prod_{n=1}^{N}p_n\right)^{1/N}$$
- BLEU는 동의어를 고려하지 않으며, 유도된 식의 많은 단계에 허술함이 존재한다.
- BLEU의 단점에 대한 참고 문헌:
[Evaluating Text Output in NLP: BLEU at Your Own Risk](https://towardsdatascience.com/evaluating-text-output-in-nlp-bleu-at-your-own-risk-e8609665a213)
- 텍스트가 토큰화되어야 하며, 토큰화 방법이 다르면 BLEU score도 달라질 수 있다.
- `SacreBLEU`: 토큰화 단계를 내장시킴 -> 벤치마킹에서 선호되는 방법

In [ ]:
# 영문판 원본의 소스 코드 ==>  실행되나, 나중에 evaluate() 때문에 오류
# from datasets import load_metric

# bleu_metric = load_metric("sacrebleu")

In [ ]:
# 번역판 소스코드
!pip install evaluate
import evaluate
bleu_metric = evaluate.load("sacrebleu")

In [ ]:
import pandas as pd
import numpy as np

bleu_metric.add(
    prediction="the the the the the the", reference=["the cat is on the mat"])
results = bleu_metric.compute(smooth_method="floor", smooth_value=0)
results["precisions"] = [np.round(p, 2) for p in results["precisions"]]
pd.DataFrame.from_dict(results, orient="index", columns=["Value"])

- `reference`: list를 전달하는 이유는 참조 문장이 여러개 일 수 있기 때문
- `smooth_method`: zero precision 방지를 위한 smoothing 방법 지정("floor"이면 smooth_value=0.1 이 default)
- `smooth_value`: n-gram이 하나도 없을 때 precision이 0 이되는 것을 방지하기 위함(여기서는 0 으로 할당하여 smoothing 기능을 끔)

In [ ]:
bleu_metric.add(
    prediction="the cat is on mat", reference=["the cat is on the mat"])
results = bleu_metric.compute(smooth_method="floor", smooth_value=0)
results["precisions"] = [np.round(p, 2) for p in results["precisions"]]
pd.DataFrame.from_dict(results, orient="index", columns=["Value"])

### 6.4.2 ROUGE
- Lin, Chin-Yew. "Rouge: A package for automatic evaluation of summaries." Text summarization branches out. 2004.
- 번역에서는 가능하고 적절한 모든 단어를 포함하는 번역 보다는 정확도(Precision)가 높은 번역이 선호되므로 기계번역에서는 `BLEU`가 많이 사용된다.
- Summarization 같은 작업에서는 생성된 텍스트에 모든 중요한 정보가 포함되기를 원하므로, 재현율(Recall)이 높기를 원하며, 이런 경우 `ROUGE`가 사용된다.
- BLEU: 후보 문장의 n-gram이 참조 문장에 얼마나 많이 등장하는지 확인
- ROUGE: 참조 문장의 n-gram이 후보 문장에 얼마나 많이 등장하는지 확인 <br>
<br>
$$\mathrm{ROUGE}_N=\frac{\sum_{snt' \in C}\sum_{n\mathrm{gram} \in snt'}Count_{\mathrm{match}}(n\mathrm{gram})}{\sum_{snt' \in C}\sum_{n\mathrm{gram} \in snt'}Count(n\mathrm{gram})}$$
<br>
- ROUGE-L: 가장 긴 공통 부분 시퀀스(LCS: Longest Common Subsequence)를 측정하는 지표
- 두 샘플을 비교할 때 긴 문장이 유리하므로 어떤 식으로든 정규화가 필요하다. 이를 위해 F-score 같은 방식으로 $F_{LCS}$를 사용한다.
  - m: 참조 문장의 길이
  - n: 후보 문장의 길이
$$R_{LCS}=\frac{LCS(X, Y)}{m}$$
$$P_{LCS}=\frac{LCS(X, Y)}{n}$$

$$F_{LCS}=\frac{{(1+\beta^2)R_{LCS}P_{LCS}}}{R_{LCS}+\beta^2P_{LCS}}, \mathrm{\qquad where}\quad \beta=\frac{P_{LCS}}{R_{LCS}}$$  

🤗Datasets에서 제공하는 ROUGE 점수 종류:
- ROUGE-L: 문장마다 점수를 계산해서 요약에 대해 평균한 점수
- ROUGE-Lsum: 전체 요약에 대해 직접 계산한 점수

In [ ]:
rouge_metric = evaluate.load("rouge")

In [ ]:
reference = dataset["train"][1]["highlights"]
records = []
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

for model_name in summaries:
    rouge_metric.add(prediction=summaries[model_name], reference=reference)
    score = rouge_metric.compute()
    rouge_dict = dict((rn, score[rn]) for rn in rouge_names)
    records.append(rouge_dict)
pd.DataFrame.from_records(records, index=summaries.keys())

PEGASUS 논문에 따르면, CNN/DaylyMail 데이터셋에서 T5 보다 뛰어나며, 적어도 BART에 견줄만 하다고 주장. PEGASUS 논문의 결과가 재현되는지 살펴보자.

## 6.5 CNN/DailyMail 데이터셋에서 PEGASUS 평가하기

노트북 중간부터 실행하는 경우에만 다음 Cell을 실행할 것!

In [ ]:
# 이 셀은 노트북 중간부터 실행하기 위한 것입니다. 실행하려면 주석처리를 삭제하자
import matplotlib.pyplot as plt
import pandas as pd
from datasets import load_dataset
import evaluate
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# "cnn_dailymail" 데이터셋 다운로드 에러가 발생할 경우 대신 "ccdv/cnn_dailymail"을 사용하세요.
dataset = load_dataset("ccdv/cnn_dailymail", version="3.0.0")
rouge_metric = evaluate.load("rouge", cache_dir=None)
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

- 이제 모델을 적절히 평가할 요소를 모두 갖췄다.
- 우선 처음 세 문장을 사용하는 baseline model의 성능부터 평가하자.

In [ ]:
def evaluate_summaries_baseline(dataset, metric,
                                column_text="article",
                                column_summary="highlights"):
    summaries = [three_sentence_summary(text) for text in dataset[column_text]]
    metric.add_batch(predictions=summaries,
                     references=dataset[column_summary])
    score = metric.compute()
    return score

- CNN/DaylyMail 데이터셋의 테스트셋의 샘플수 = ~10,000 개
- 요약 생성에는 많은 시간이 걸린다.
  - 생성되는 모든 토큰이 forward pass를 거쳐야 하므로
  - 샘플마다 100개의 토큰을 생성하려면 100 x 10000 = 1백만번의 forward pass를 거쳐야 함.
- 원래 저자의 책에는 시간을 줄이기 위해 테스트셋에서 1000개을 샘플링하여 사용.
- 실습에서는 시간을 더 줄이기 위해 200개만 사용하기로 함

우선 baseline 모델 평가.

In [ ]:
test_sampled = dataset["test"].shuffle(seed=42).select(range(200))  # 1000을 200으로 수정 - kikim6114

score = evaluate_summaries_baseline(test_sampled, rouge_metric)
rouge_dict = dict((rn, score[rn]) for rn in rouge_names)
pd.DataFrame.from_dict(rouge_dict, orient="index", columns=["baseline"]).T

PEGASUS 모델 평가를 위한 함수:

In [ ]:
from tqdm import tqdm
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

def chunks(list_of_elements, batch_size):
    """list_of_elements로부터 batch_size 크기의 청크를 연속적으로 생성합니다"""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]

def evaluate_summaries_pegasus(dataset, metric, model, tokenizer,
                               batch_size=16, device=device,
                               column_text="article",
                               column_summary="highlights"):
    article_batches = list(chunks(dataset[column_text], batch_size))
    target_batches = list(chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):

        inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                        padding="max_length", return_tensors="pt")

        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device),
                         length_penalty=0.8, num_beams=8, max_length=128)  # 논문과 동일한 매개변수 값 사용

        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                clean_up_tokenization_spaces=True)
               for s in summaries]
        decoded_summaries = [d.replace("<n>", " ") for d in decoded_summaries]
        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    score = metric.compute()
    return score

이제 seq2seq 생성 작업에 사용되는 `AutoModelForSeq2SeqLM`을 사용하여 모델을 다시 로드하여 평가해본다.

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_ckpt = "google/pegasus-cnn_dailymail"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)
score = evaluate_summaries_pegasus(test_sampled, rouge_metric,
                                   model, tokenizer, batch_size=8)
rouge_dict = dict((rn, score[rn]) for rn in rouge_names)
pd.DataFrame(rouge_dict, index=["pegasus"])

## 6.6 요약 모델 훈련하기
- 앞의 내용을 활용해 Text Summarization 모델을 직접 훈련해보자.
- 삼성이 만든 SAMsum 데이터셋을 사용: 고객센터 대화 내용

In [ ]:
dataset_samsum = load_dataset("samsum")
split_lengths = [len(dataset_samsum[split])for split in dataset_samsum]

print(f"분할 크기: {split_lengths}")
print(f"특성: {dataset_samsum['train'].column_names}")
print("\n대화:")
print(dataset_samsum["test"][0]["dialogue"])
print("\nSummary:")
print(dataset_samsum["test"][0]["summary"])

### 6.6.1 SAMSum에서 PEGASUS 평가하기
- PUGASUS로 요약 파이프라인을 실행해 어떻게 출력되는지 보자.

In [ ]:
pipe_out = pipe(dataset_samsum["test"][0]["dialogue"])
print("요약:")
print(pipe_out[0]["summary_text"].replace(" .<n>", ".\n"))

In [ ]:
SAMsum에서의 요약은 CNN/DailyMail에서보다 추상적이다.

In [ ]:
score = evaluate_summaries_pegasus(dataset_samsum["test"], rouge_metric, model,
                                   tokenizer, column_text="dialogue",
                                   column_summary="summary", batch_size=8)

rouge_dict = dict((rn, score[rn]) for rn in rouge_names)
pd.DataFrame(rouge_dict, index=["pegasus"])

결과가 훌륭하지는 않지만 CNN/DailyMail 데이터셋이 SAMsum 과 크게 다르기 때문에 어느 정도 예상할 수 있는 결과다.
훈련 전에 파이프라인을 준비하면 두 가지 잇점이 있다:
- 훈련이 성공적인지 바로 평가가 가능
- 기준점이 수립됨  

이 데이터셋에서 모델을 미세조정하면 ROUGE 점수가 바로 향상되어야 한다. 그렇지 않으면 훈련과정에 문제가 있는 것이다.

### 6.6.2 PEGASUS 미세 튜닝하기

In [ ]:
d_len = [len(tokenizer.encode(s)) for s in dataset_samsum["train"]["dialogue"]]
s_len = [len(tokenizer.encode(s)) for s in dataset_samsum["train"]["summary"]]

fig, axes = plt.subplots(1, 2, figsize=(10, 3.5), sharey=True)
axes[0].hist(d_len, bins=20, color="C0", edgecolor="C0")
axes[0].set_title("Dialogue Token Length")
axes[0].set_xlabel("Length")
axes[0].set_ylabel("Count")
axes[1].hist(s_len, bins=20, color="C0", edgecolor="C0")
axes[1].set_title("Summary Token Length")
axes[1].set_xlabel("Length")
plt.tight_layout()
plt.show()

- Trainer를 위한 data collator를 만들자
  - 대화 최대 길이 = 1024
  - 요약 최대 길이 = 128

In [ ]:
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch["dialogue"], max_length=1024,
                                truncation=True)

    with tokenizer.as_target_tokenizer():  # 번역판에서는 context manager with~를 사용하지 않음
        target_encodings = tokenizer(text_target=example_batch["summary"], max_length=128,
                                    truncation=True)

    return {"input_ids": input_encodings["input_ids"],
            "attention_mask": input_encodings["attention_mask"],
            "labels": target_encodings["input_ids"]}

dataset_samsum_pt = dataset_samsum.map(convert_examples_to_features,
                                       batched=True)
columns = ["input_ids", "labels", "attention_mask"]
dataset_samsum_pt.set_format(type="torch", columns=columns)

`with tokenizer.as_target_tokenizer()`: 인코더와 디코더의 tokenizer를 구별하는 것이 중요할 때 사용

In [ ]:
# 티처 포싱(teacher forcing)
# 텍스트 생성을 위한 디코더 입력과 레이블의 정렬
text = ['PAD','Transformers', 'are', 'awesome', 'for', 'text', 'summarization']
rows = []
for i in range(len(text)-1):
    rows.append({'step': i+1, 'decoder_input': text[:i+1], 'label': text[i+1]})
pd.DataFrame(rows).set_index('step')

In [ ]:
from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='pegasus-samsum', num_train_epochs=1, warmup_steps=500,
    per_device_train_batch_size=1, per_device_eval_batch_size=1,
    weight_decay=0.01, logging_steps=10, push_to_hub=True,
    evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
    gradient_accumulation_steps=16)

- `gradient_accumulation_steps=16`: 모델이 크면 batch_size=1 로 하는데, batch가 너무 작으면 수렴하지 않는다.
- 이 문제를 해결하기 위한 기법으로서, 16회 그래디언트가 누적되어 충분히 커지면 최적화 단계가 진행됨
- 학습 속도는 느려지지만 GPU 메모리가 많이 절약됨

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
trainer = Trainer(model=model, args=training_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["train"],
                  eval_dataset=dataset_samsum_pt["validation"])

In [ ]:
trainer.train()
score = evaluate_summaries_pegasus(
    dataset_samsum["test"], rouge_metric, trainer.model, tokenizer,
    batch_size=2, column_text="dialogue", column_summary="summary")

rouge_dict = dict((rn, score[rn]) for rn in rouge_names)
pd.DataFrame(rouge_dict, index=[f"pegasus"])

In [ ]:
trainer.push_to_hub("Training complete!")

### 6.6.3 대화 요약 생성하기

Loss 값과 ROUGE 점수를 보면 CNN/DailyMail에서만 훈련한 원래 모델보다 크게 향상된 것 같다.

In [ ]:
import transformers
transformers.logging.set_verbosity_error()

In [ ]:
gen_kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length": 128}
sample_text = dataset_samsum["test"][0]["dialogue"]
reference = dataset_samsum["test"][0]["summary"]
# `haesun`를 자신의 허브 사용자 이름으로 바꾸세요.
pipe = pipeline("summarization", model="haesun/pegasus-samsum")

print("대화:")
print(sample_text)
print("\n참조 요약:")
print(reference)
print("\n모델 요약:")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])

- 참조 문장과 훨씬 더 비슷해졌다. 모델이 그냥 문장을 추출하지 않고 대화를 합성해서 요약을 만드는 법을 배운것 같다.
- 실제 대화 입력에서 이 모델은 얼마나 잘 작동할까?

In [ ]:
custom_dialogue = """\
Thom: Hi guys, have you heard of transformers?
Lewis: Yes, I used them recently!
Leandro: Indeed, there is a great library by Hugging Face.
Thom: I know, I helped build it ;)
Lewis: Cool, maybe we should write a book about it. What do you think?
Leandro: Great idea, how hard can it be?!
Thom: I am in!
Lewis: Awesome, let's do it together!
"""
print(pipe(custom_dialogue, **gen_kwargs)[0]["summary_text"])

## 6.7 결론

- Text Summarization은 sentiment analysis, NER, QA 같은 분류 작업들보다 몇가지 특수한 어려움이 있다.
- BLEU나 ROUGE가 있지만 역시 사람의 판단이 가장 좋은 척도다
- Summarization 모델로 작업을 수행할 때, 문맥 크기 보다는 긴 텍스트를 요약하는 방법에 의문을 갖게 된다.
- OpenAI는 긴 문서에 반복적으로 모델을 적용하고 사람의 피드백을 반복 루프에 추가하여 summarization task의 scale을 확장했다.
- Wu, Jeff, et al. "Recursively summarizing books with human feedback." arXiv preprint arXiv:2109.10862 (2021).